In [137]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [138]:
data=pd.read_csv("datasets/titanic/train.csv")

In [139]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [140]:
sample=data.drop(["PassengerId","Name","Ticket","Fare","Cabin","Embarked"],axis=1)

In [141]:
sample.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [142]:
sample["Sex"]=LabelEncoder().fit_transform(sample["Sex"])

In [143]:
sample[:5]

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [144]:
processed=sample.dropna()

In [145]:
model=processed.groupby(["Pclass","Sex"]).mean()[["Age"]].reset_index()  ###gourpby work with mean

In [146]:
model

,Pclass,Sex,Age
0,1,0,34.611765
1,1,1,41.281386
2,2,0,28.722973
3,2,1,30.740707
4,3,0,21.750000
5,3,1,26.507589


In [147]:
unpro=sample.loc[sample["Age"].isna()]

In [148]:
def Agefill(row):
    row["Age"]=model.loc[(row["Pclass"]==model["Pclass"]) & (row["Sex"]==model["Sex"])].describe().loc["mean","Age"]
    return row

In [149]:
processed1=unpro.apply(Agefill,axis=1)

In [150]:
total_data=pd.concat([processed,processed1])

In [151]:
total_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0.0,3.0,1.0,22.0,1.0,0.0
1,1.0,1.0,0.0,38.0,1.0,0.0
2,1.0,3.0,0.0,26.0,0.0,0.0
3,1.0,1.0,0.0,35.0,1.0,0.0
4,0.0,3.0,1.0,35.0,0.0,0.0


In [152]:
## Decision Tree

In [153]:
X=total_data.drop(["Survived"],axis=1)
y=total_data["Survived"]

In [155]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [161]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [162]:
model=DecisionTreeClassifier()

In [164]:
model.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [165]:
model.score(X_test,y_test)

0.7932203389830509

In [166]:
def entropy(data):
    full,count=np.unique(data,return_counts=True)
    p=count/len(data)
    return np.sum(-p*np.log(p))

In [174]:
def info_gain(X,y,label):
    
    x_col=X[label]
    pivot=np.mean(x_col)
    
    left_index=x_col<pivot
    right_index=x_col>pivot
    
    y_left , y_right =y[left_index] ,y[right_index]
    
    ent=entropy(y)
    left_ent=entropy(y_left)
    right_ent=entropy(y_right)
    
    left_p=len(y_left)/len(y)
    right_p=1-left_p
    
    return ent - left_ent*left_p - right_ent*right_p , pivot

In [175]:
for col in X.columns:
    print(col,info_gain(X,y,col))

Pclass (0.05253613916173083, 2.308641975308642)
Sex (0.15087048925218172, 0.6475869809203143)
Age (0.0007663481843047659, 29.318642716644238)
SibSp (0.006643498134917131, 0.5230078563411896)
Parch (0.010661126611803073, 0.38159371492704824)
